<a href="https://colab.research.google.com/github/ClaytonsCreations/AWS_BigData/blob/main/watchReviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Ign:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:12 http://archive.ubuntu.com/ubuntu bionic-backpor

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-02-23 02:31:51--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.12MB/s    in 0.2s    

2022-02-23 02:31:52 (5.12 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url ="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Watches_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
watches = spark.read.csv(SparkFiles.get("amazon_reviews_us_Watches_v1_00.tsv.gz"), sep="\t", header=True)

# Show DataFrame
watches.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    3653882|R3O9SGZBVQBV76|B00FALQ1ZC|     937001370|Invicta Women's 1...|         Watches|          5|            0|          0|   N|                Y|          Five Stars|Absolutely love t...| 2015-08-31|
|         US|   14661224| RKH8BNC3L5DLF|B00D3RGO20|     484010722|Kenneth Cole New ...|         Watches|          5|    

In [5]:
watches.count()

960872

In [6]:
watches = watches.dropna()

In [7]:
watches = watches.dropDuplicates()

In [8]:
watches.count()

960679

In [9]:
watches.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: string (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- helpful_votes: string (nullable = true)
 |-- total_votes: string (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [10]:
from pyspark.sql.types import StringType, DateType, FloatType, IntegerType

watches = watches.withColumn("customer_id",watches["customer_id"].cast(IntegerType())) \
    .withColumn("star_rating",watches["star_rating"].cast(IntegerType())) \
    .withColumn("product_parent",watches["product_parent"].cast(IntegerType())) \
    .withColumn("review_date",watches["review_date"].cast(DateType())) \
    .withColumn("helpful_votes",watches["helpful_votes"].cast(IntegerType())) \
    .withColumn("total_votes",watches["total_votes"].cast(IntegerType())) 
watches.printSchema()


root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: date (nullable = true)



In [11]:
review_id_table = watches.select(['review_id', 'customer_id', 'product_id', 'product_parent', 'review_date'])
review_id_table.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R101G33FIEDHQG|   22594238|B000PSN090|     425533441| 2015-02-08|
|R10339L4K4LX3C|   20301275|B00LTQTH58|     732935288| 2014-11-18|
|R1073CM97QGOPP|   32190651|B004V23YV0|     921921436| 2015-07-06|
|R107KYD2GYG4DK|    1424324|B004Y4CMCM|     452126791| 2014-07-05|
|R10AYHA3991CS9|   26642389|B00J0SSATQ|     682900500| 2014-10-28|
|R10C59FCUFOZPG|   15168495|B005FMYSWO|     824130779| 2012-03-07|
|R10DZLWV4QYCPI|    2239995|B000FVC5LG|     918169894| 2014-08-21|
|R10E5DEJ235Z19|    5378758|B003IHFGBK|     857934958| 2014-02-08|
|R10E6QP8RITUIA|    8495726|B00DI315GE|     734054846| 2015-03-23|
|R10F6W35V8C1ER|   25578677|B006076T5M|     832352606| 2013-04-10|
|R10G5F537HMGLR|   12806455|B00CJXMLNK|     908427132| 2014-01-06|
|R10GL8NCNQU8M7|   15231446|B00I9I70BM|     419690985| 2015-04

In [12]:
products = watches.select(['product_id', 'product_title'])
products.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B000PSN090|Nautica Men's N14...|
|B00LTQTH58|Sotijobs Unisex T...|
|B004V23YV0|Seiko 5 Sport Aut...|
|B004Y4CMCM|Casio Women's LTP...|
|B00J0SSATQ|Armitron Sport Un...|
|B005FMYSWO|Invicta Men's 092...|
|B000FVC5LG|Invicta Women's 2...|
|B003IHFGBK|Nixon Unisex Time...|
|B00DI315GE|Stuhrling Origina...|
|B006076T5M|ShoppeWatch Skele...|
|B00CJXMLNK|TW Steel Pilot Ti...|
|B00I9I70BM|Casio Men's W-S22...|
|B0030DFF90|Anne Klein Modern...|
|B001P80F1U|Invicta Women's 5...|
|B001SN7RYU|Invicta Men's 721...|
|B00R5Z4RIY|Lucien Piccard Me...|
|B0035KKR82|Fossil Ch2601p De...|
|B00E4PQE4S|Armitron Sport Me...|
|B00J5EINZG|Versace Men's VDB...|
|B006SICNWG|ShoppeWatch Hand ...|
+----------+--------------------+
only showing top 20 rows



In [22]:
products = products.dropDuplicates()

In [13]:
customers = watches.groupBy('customer_id').agg({'customer_id': 'count'})
customers = customers.withColumnRenamed('count(customer_id)', 'customer_count')
customers.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|    2252188|             1|
|   52373157|             3|
|    2594156|             2|
|   13373248|             1|
|   20671851|            24|
|   45153737|             5|
|    2891243|             2|
|   45938967|             1|
|   39355523|             1|
|   29954480|             1|
|   45726037|             2|
|   12665682|             1|
|   12550672|             7|
|   22426070|             1|
|   13309307|             1|
|   11441161|             2|
|   25694626|             1|
|   12799553|             1|
|   24740812|             1|
|   12536485|             3|
+-----------+--------------+
only showing top 20 rows



In [14]:
vine_table = watches.select(['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine'])
vine_table.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R101G33FIEDHQG|          3|            0|          0|   N|
|R10339L4K4LX3C|          4|            1|          2|   N|
|R1073CM97QGOPP|          5|            0|          0|   N|
|R107KYD2GYG4DK|          5|            0|          0|   N|
|R10AYHA3991CS9|          5|            2|          2|   N|
|R10C59FCUFOZPG|          5|            3|          3|   N|
|R10DZLWV4QYCPI|          5|            0|          0|   N|
|R10E5DEJ235Z19|          4|            1|          2|   N|
|R10E6QP8RITUIA|          5|            0|          0|   N|
|R10F6W35V8C1ER|          5|            0|          0|   N|
|R10G5F537HMGLR|          3|           26|         34|   N|
|R10GL8NCNQU8M7|          3|            0|          0|   N|
|R10LINWDMEJ36W|          5|            0|          0|   N|
|R10LY9CC9X0LFU|          5|            

In [15]:
mode = "append"
jdbc_url="jdbc:postgresql://<endpoint>.amazonaws.com:5432/watches"
config = {"user":"root", "password": <password>, "driver":"org.postgresql.Driver"}

In [ ]:
review_id_table.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [23]:
products.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [18]:
customers.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [19]:
vine_table.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)